In [ ]:
# ===============================
# Cell 1 : paths  +  quick sanity
# ===============================
import os, glob, json, random
from pathlib import Path
import pandas as pd
from collections import Counter
import torch

# -------- 1. Base paths --------
BASE = Path('/kaggle/input/identity-employees-in-surveillance-cctv')

# detect whether train lives under “dataset/train” or “dataset/dataset/train”
possible_roots = [BASE / 'dataset', BASE / 'dataset' / 'dataset']
for candidate in possible_roots:
    if (candidate / 'train' / 'images').exists():
        ROOT = candidate
        break
else:
    raise FileNotFoundError(f"Could not locate train/images under {possible_roots}")

# define train / unseen-test paths
TRAIN_IMG_DIR = ROOT / 'train' / 'images'
LABELS_CSV    = ROOT / 'train' / 'labels.csv'
UNSEEN_ROOT   = BASE / 'dataset_unseen'
TEST_IMG_DIR  = UNSEEN_ROOT / 'unseen_test' / 'images'

# -------- 2. Sanity checks --------
for p in (TRAIN_IMG_DIR, TEST_IMG_DIR, LABELS_CSV):
    assert p.exists(), f"Missing: {p}"

print("Train images :", len(list(TRAIN_IMG_DIR.glob('*.jpg'))))
print(" Test images :", len(list(TEST_IMG_DIR.glob('*.jpg'))))

# -------- 3. Read & CLEAN labels_df --------
labels_df = pd.read_csv(LABELS_CSV)

# ✨ NEW: drop rows pointing to images that are gone (e.g. face_1147.jpg)
mask_exists = labels_df['filename'].apply(lambda f: (TRAIN_IMG_DIR / f).exists())
labels_df   = labels_df[mask_exists].reset_index(drop=True)

print(f"After dropping missing images → {len(labels_df):,} rows remain")
display(labels_df.head())

# -------- 4. Quick counts (optional) --------
emp_counts = Counter(labels_df['emp_id'])
print(f"Unique employees: {len(emp_counts)}")
print("Five smallest classes:", emp_counts.most_common()[-5:])

# -------- 5. GPU check (optional) --------
print("CUDA available :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name       :", torch.cuda.get_device_name(0))


In [ ]:
# ================================================================
# Cell A : extract 500 JPG frames / employee from reference video
#          + save them in EXTRA_DIR  +  create new_rows list
# ================================================================
import cv2, numpy as np
from pathlib import Path

EXTRA_DIR = Path('/kaggle/working/tmp_ref_frames')
EXTRA_DIR.mkdir(parents=True, exist_ok=True)


new_rows = []                 # ← will be concatenated with labels_df in Cell 2
SAMPLES  = 500                # number of evenly-spaced frames per video

for emp_dir in (ROOT / 'reference_faces').iterdir():        # emp001, emp002 …
    vid_files = list(emp_dir.glob('*.mp4'))
    if not vid_files:
        continue                               # some employees may have only JPGs
    vid = vid_files[0]
    cap = cv2.VideoCapture(str(vid))
    if not cap.isOpened():
        print(f"⚠️ could not open {vid}")
        continue

    tot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # ----- sample SAMPLES indices evenly across the video ----------
    for f in np.linspace(0, tot - 1, SAMPLES, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, f)
        ok, frame = cap.read()
        if not ok:
            continue
        fname = f'{emp_dir.name}_{f:04d}.jpg'               # emp001_0003.jpg …
        cv2.imwrite(str(EXTRA_DIR / fname), frame)
        new_rows.append({'filename': fname, 'emp_id': emp_dir.name})

    cap.release()

print(f"✅ Saved {len(new_rows):,} extra frames to {EXTRA_DIR}")


In [ ]:
# ================================================================
# Cell 2 : dataset, transforms, stratified split (CCTV hold-out)
# ================================================================
import subprocess, sys, importlib.util, numpy as np, torch, random
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
from pathlib import Path
from PIL import Image
import pandas as pd
from io import BytesIO

# 0️⃣  Install timm (once per kernel) --------------------------------
if importlib.util.find_spec("timm") is None:
    subprocess.run([sys.executable, "-m", "pip", "-q", "install",
                    "timm", "torchmetrics", "--no-progress"])

# -------------------------------------------------------------------
# 1️⃣  Build master labels_df
#     (Cell 1 already loaded + cleaned labels_df, so we start there)
# -------------------------------------------------------------------
labels_df = labels_df.copy()                 # make local copy of clean df

# 1.0  Append the 500-frame video samples made in Cell A
extra_df  = pd.DataFrame(new_rows)           # list produced in Cell A
labels_df = pd.concat([labels_df, extra_df], ignore_index=True)

# 1.1  Append static reference JPGs as “photo” rows
static_rows = []
for emp_dir in (ROOT / 'reference_faces').iterdir():
    for jpg in emp_dir.glob('*.jpg'):
        static_rows.append({'filename': jpg.name,
                            'emp_id':   emp_dir.name})
        # symlink each static JPG into EXTRA_DIR so IMG_ROOTS can find it
        link = EXTRA_DIR / jpg.name
        if not link.exists():
            link.symlink_to(jpg)
labels_df = pd.concat([labels_df, pd.DataFrame(static_rows)],
                      ignore_index=True)

# 1.2  Final safety filter — keep only rows whose file exists
IMG_ROOTS = [TRAIN_IMG_DIR, EXTRA_DIR]       # search order for images
labels_df = labels_df[
    labels_df['filename'].apply(
        lambda f: any((root / f).exists() for root in IMG_ROOTS)
    )
].reset_index(drop=True)
print(f"After drop-missing ➜ {len(labels_df):,} rows")

# -------------------------------------------------------------------
# 2️⃣  Encode employee IDs → int labels
# -------------------------------------------------------------------
employee_ids = sorted(labels_df.emp_id.unique())
emp2idx = {e: i for i, e in enumerate(employee_ids)}
labels_df["label_idx"] = labels_df.emp_id.map(emp2idx)

# 3️⃣  Tag each row as CCTV vs photo
labels_df["source"] = labels_df.filename.apply(
    lambda fn: "photo" if (EXTRA_DIR / fn).exists() else "cctv"
)

# -------------------------------------------------------------------
# 4️⃣  Transforms
# -------------------------------------------------------------------
class JpegCompression:                       # PIL-only artefact
    def __init__(self, quality=(30, 60)):
        self.qmin, self.qmax = quality
    def __call__(self, img: Image.Image):
        buf = BytesIO()
        q   = random.randint(self.qmin, self.qmax)
        img.save(buf, format="JPEG", quality=q)
        buf.seek(0)
        return Image.open(buf)

train_tfms = T.Compose([
    T.RandomResizedCrop(224, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(0.5),
    T.ColorJitter(0.2, 0.2, 0.2, 0.1),
    T.RandomRotation(15),
    T.RandomPerspective(0.3, p=0.2),
    T.RandomApply([T.GaussianBlur(3)], p=0.2),
    T.RandomApply([JpegCompression((30, 60))], p=0.3),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225]),
    T.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3)),
])

val_tfms = T.Compose([
    T.Resize(256), T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225]),
])

# -------------------------------------------------------------------
# 5️⃣  Dataset (multi-root lookup)
# -------------------------------------------------------------------
class FaceDS(Dataset):
    def __init__(self, df, tfm):
        self.df   = df.reset_index(drop=True)
        self.tfm  = tfm
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        for root in IMG_ROOTS:
            p = root / row.filename
            if p.exists():
                img = Image.open(p).convert("RGB")
                break
        else:
            raise FileNotFoundError(f"Missing {row.filename}")
        return self.tfm(img), row.label_idx

full_ds = FaceDS(labels_df, train_tfms)

# -------------------------------------------------------------------
# 6️⃣  Stratified 10 % hold-out on CCTV (keep all photos in train)
# -------------------------------------------------------------------
cctv_idx  = labels_df.index[labels_df.source == "cctv"].to_numpy()
cctv_lbls = labels_df.label_idx.loc[cctv_idx].to_numpy()

# keep singleton classes entirely in train
from collections import Counter
cnt = Counter(cctv_lbls)
singleton_cls = [cls for cls, n in cnt.items() if n == 1]
mask_single   = np.isin(cctv_lbls, singleton_cls)
single_idx    = cctv_idx[mask_single]
multi_idx     = cctv_idx[~mask_single]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=42)
train_multi, val_multi = next(sss.split(multi_idx,
                                        labels_df.label_idx.loc[multi_idx]))
train_cctv = np.concatenate([single_idx, multi_idx[train_multi]])
val_cctv   = multi_idx[val_multi]

photo_idx  = labels_df.index[labels_df.source == "photo"].to_numpy()

train_idx = np.concatenate([train_cctv, photo_idx])
val_idx   = val_cctv

# -------------------------------------------------------------------
# 7️⃣  DataLoaders
# -------------------------------------------------------------------
train_ds = Subset(full_ds, train_idx)
val_ds   = Subset(FaceDS(labels_df.loc[val_idx], val_tfms),
                  np.arange(len(val_idx)))

BATCH = 32
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                      num_workers=4, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                      num_workers=4, pin_memory=True)

print(f"Train images: {len(train_ds):,} | Val images: {len(val_ds):,}")
print("Smallest training classes:",
      Counter(labels_df.label_idx.loc[train_idx]).most_common()[-5:])


In [ ]:
# quick visual sanity-check
import matplotlib.pyplot as plt
sample_img, _ = full_ds[0]              # any sample after transforms
plt.imshow(sample_img.permute(1,2,0).numpy()*0.229+0.485)
plt.axis('off'); plt.title('Augmentation example');


In [ ]:
# ================================================================
# Cell 3 : EfficientNet-B0 + ArcFace fine-tune (50 epochs)
# ================================================================
import timm, torch, math, numpy as np
from torch import nn
from torchmetrics.classification import MulticlassAccuracy

device       = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES  = len(employee_ids)                 # set in Cell 2
EPOCHS       = 50
CKPT         = '/kaggle/working/effb0_arcface.pt'

# ── 1️⃣  Backbone + 512-D projection ────────────────────────────
backbone = timm.create_model(
    'efficientnet_b0', pretrained=True, num_classes=0, global_pool='avg'
).to(device)
proj = nn.Linear(backbone.num_features, 512, bias=False).to(device)

# ── 2️⃣  ArcFace additive-margin head ───────────────────────────
class ArcFace(nn.Module):
    def __init__(self, in_f, out_f, s=30.0, m=0.50):
        super().__init__()
        self.W = nn.Parameter(torch.randn(out_f, in_f))
        nn.init.xavier_uniform_(self.W)
        self.s, self.m = s, m
        self.cos_m, self.sin_m = math.cos(m), math.sin(m)
        self.th, self.mm = math.cos(math.pi - m), math.sin(math.pi - m) * m
    def forward(self, x, labels):
        x_n = nn.functional.normalize(x)
        w_n = nn.functional.normalize(self.W)
        cos = nn.functional.linear(x_n, w_n)            # cosine θ
        sin = torch.sqrt(1.0 - torch.clamp(cos**2, 0, 1))
        phi = cos * self.cos_m - sin * self.sin_m        # cos(θ+m)
        phi = torch.where(cos > self.th, phi, cos - self.mm)
        one_hot = torch.zeros_like(cos, device=x.device)
        one_hot.scatter_(1, labels.view(-1,1), 1.0)
        logits = (one_hot * phi + (1.0 - one_hot) * cos) * self.s
        return logits

arc = ArcFace(512, NUM_CLASSES).to(device)

# ── 3️⃣  Optimizer, warm-up, cosine LR ─────────────────────────
params = list(backbone.parameters()) + list(proj.parameters()) + list(arc.parameters())
opt    = torch.optim.AdamW(params, lr=3e-4, weight_decay=1e-4)
warm   = torch.optim.lr_scheduler.LinearLR(opt, start_factor=0.1, total_iters=2)
sched  = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
metric    = MulticlassAccuracy(num_classes=NUM_CLASSES, average='macro').to(device)
best_acc  = 0.0

# ── 4️⃣  Training loop ─────────────────────────────────────────
for ep in range(1, EPOCHS + 1):
    # ---- train ----
    backbone.train(); proj.train(); arc.train()
    for x, y in train_dl:                       # train_dl from Cell 2
        x, y = x.to(device), y.to(device)
        feats  = proj(backbone(x))
        logits = arc(feats, y)
        loss   = criterion(logits, y)

        opt.zero_grad(); loss.backward(); opt.step()

    # ---- scheduler ----
    (warm if ep <= 2 else sched).step()

    # ---- validation ----
    backbone.eval(); proj.eval(); metric.reset()
    with torch.no_grad():
        # pre-normalize weight matrix once per epoch
        Wn = nn.functional.normalize(arc.W, dim=1)      # [C,512]
        for x, y in val_dl:                             # val_dl from Cell 2
            x, y = x.to(device), y.to(device)
            feats = proj(backbone(x))
            fnorm = nn.functional.normalize(feats, dim=1)
            preds = torch.matmul(fnorm, Wn.T).argmax(1) # cosine scores
            metric.update(preds, y)

    acc = metric.compute().item()
    print(f"Epoch {ep:03d} | Val Macro-Acc {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        torch.save({'backbone': backbone.state_dict(),
                    'proj'    : proj.state_dict(),
                    'arc'     : arc.state_dict()}, CKPT)
        print("  🔥 new best saved")

print(f"\n▶ Best validation Macro-Accuracy: {best_acc:.4f}")


In [ ]:
# --------------------------------------------------
# after reading labels.csv  (Cell 1 or wherever)
# --------------------------------------------------
TRAIN_IMG_DIR = Path('/kaggle/input/identity-employees-in-surveillance-cctv'
                     '/dataset/dataset/train/images')

# keep only rows whose image file is really on disk
labels_df = labels_df[
    labels_df['filename'].apply(lambda f: (TRAIN_IMG_DIR / f).exists())
].reset_index(drop=True)

print(f"Filtered labels_df → {len(labels_df):,} rows remain")


In [ ]:
# ================================================================
# Cell 4 : build FIQA-filtered gallery + sweep cosine τ
# ================================================================
import sys, os, gc
import numpy as np
import torch
import torch.nn as nn
import timm
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
from torchvision import transforms as T
from insightface.app import FaceAnalysis

# ── device & base paths ─────────────────────────────────────────
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BASE   = Path('/kaggle/input/identity-employees-in-surveillance-cctv')

# ── updated dataset paths ───────────────────────────────────────
TRAIN_ROOT   = BASE / 'dataset' / 'dataset'
REF_DIR      = TRAIN_ROOT / 'reference_faces'
EXTRA_DIR = Path('/kaggle/working/tmp_ref_frames')  # updated to where frames were saved

# ── sanity-check reference_faces ────────────────────────────────
if not REF_DIR.exists():
    raise FileNotFoundError(f"Missing required directory: {REF_DIR}")

# ── check for extracted frames; skip if none ────────────────────
if EXTRA_DIR.exists() and any(EXTRA_DIR.glob('*.jpg')):
    extra_imgs = list(EXTRA_DIR.glob('*.jpg'))
    print(f"Found {len(extra_imgs)} extracted frames in {EXTRA_DIR}")
    use_extra = True
else:
    print(f"⚠️  No extracted frames in {EXTRA_DIR}, skipping video-frames step.")
    extra_imgs = []
    use_extra = False

# ── reload trained backbone + projection ─────────────────────────
CKPT     = '/kaggle/working/effb0_arcface.pt'
backbone = timm.create_model('efficientnet_b0', pretrained=False,
                             num_classes=0, global_pool='avg').to(device)
proj     = nn.Linear(backbone.num_features, 512, bias=False).to(device)

ck = torch.load(CKPT, map_location=device)
backbone.load_state_dict(ck['backbone'])
proj.load_state_dict(ck['proj'])
backbone.eval(); proj.eval()

# ── FIQA quality model ───────────────────────────────────────────
qa = FaceAnalysis(name='buffalo_s')
qa.prepare(ctx_id=0)

# ── transforms for gallery & val ─────────────────────────────────
val_tfms = T.Compose([
    T.Resize(256), T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# ── build raw gallery (static JPEGs + optional video frames) ────
ref_paths, ref_labels, ref_embs = [], [], []

# 1) high-quality reference photos
for emp_dir in REF_DIR.iterdir():
    for jpg in emp_dir.glob('*.jpg'):
        img = Image.open(jpg).convert('RGB')
        ref_paths.append(jpg)
        ref_labels.append(emp_dir.name)
        with torch.no_grad():
            emb = proj(backbone(val_tfms(img).unsqueeze(0).to(device))).squeeze().cpu()
        ref_embs.append(emb)

# 2) optionally include extracted video frames
if use_extra:
    for jpg in extra_imgs:
        img = Image.open(jpg).convert('RGB')
        ref_paths.append(jpg)
        ref_labels.append(jpg.stem.split('_')[0])
        with torch.no_grad():
            emb = proj(backbone(val_tfms(img).unsqueeze(0).to(device))).squeeze().cpu()
        ref_embs.append(emb)

print("Raw gallery size :", len(ref_embs))

# ── FIQA filtering: drop bottom 30% by det_score ────────────────
scores = []
for p in tqdm(ref_paths, desc="FIQA scoring"):
    arr   = np.array(Image.open(p).convert('RGB'))
    faces = qa.get(arr)
    scores.append(faces[0].det_score if faces else 0.0)

threshold = np.percentile(scores, 20)
keep_ix   = [i for i, s in enumerate(scores) if s >= threshold]

ref_embs   = torch.stack([ref_embs[i] for i in keep_ix])
ref_labels = [ref_labels[i]     for i in keep_ix]
ref_norm   = nn.functional.normalize(ref_embs, dim=1)
print("Filtered gallery:", ref_norm.shape)

# ── embed validation CCTV set & sweep τ ─────────────────────────
val_embs, val_true = [], []
idx2emp = {v: k for k, v in emp2idx.items()}

for xb, yb in tqdm(val_dl, desc="val embeddings"):
    xb = xb.to(device)
    with torch.no_grad():
        feats = backbone(xb)
        embs  = proj(feats).cpu()
    val_embs.append(embs)
    val_true.extend([idx2emp[int(i)] for i in yb])

val_embs = torch.cat(val_embs, dim=0)

def macro_acc(true, pred):
    classes = sorted(set(true))
    return np.mean([
        np.mean([t == p for t, p in zip(true, pred) if t == c])
        for c in classes
    ])

best_tau, best_ma = None, -1
for τ in np.linspace(0.30, 0.55, 51):
    preds = []
    for e in val_embs:
        sims = ref_norm @ nn.functional.normalize(e, dim=0)
        j    = sims.argmax().item()
        preds.append(ref_labels[j] if sims[j] >= τ else "unknown")
    m = macro_acc(val_true, preds)
    print(f"τ={τ:.2f} → Val Macro-Acc {m:.4f}")
    if m > best_ma:
        best_ma, best_tau = m, τ

print(f"\n▶ Best τ = {best_tau:.2f}  (Val Macro-Acc = {best_ma:.4f})")

# ── save gallery + threshold for Cell 5 ─────────────────────────
torch.save({
    'ref_norm':   ref_norm,
    'ref_labels': ref_labels,
    'best_tau':   best_tau
}, '/kaggle/working/gallery.pt')

# cleanup
gc.collect()


In [ ]:
# ================================================================
# Cell 4B : grid-search (P_TH, τ) on validation set
# ================================================================

import numpy as np
import torch
import torch.nn as nn
import timm
from itertools import product

device      = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = len(employee_ids)

# ── 1️⃣ Redefine ArcFace to match your checkpoint keys ────────────
class ArcFace(nn.Module):
    def __init__(self, in_f, out_f, s=30.0, m=0.50):
        super().__init__()
        self.W = nn.Parameter(torch.randn(out_f, in_f))
        nn.init.xavier_uniform_(self.W)
        self.s, self.m = s, m
        self.cos_m, self.sin_m = np.cos(m), np.sin(m)
        self.th, self.mm = np.cos(np.pi - m), np.sin(np.pi - m) * m

    def forward(self, x, y):
        x_n = nn.functional.normalize(x)
        w_n = nn.functional.normalize(self.W)
        cos = nn.functional.linear(x_n, w_n)
        sin = torch.sqrt(1.0 - torch.clamp(cos**2, 0, 1))
        phi = torch.where(cos > self.th,
                          cos * self.cos_m - sin * self.sin_m,
                          cos - self.mm)
        one_hot = torch.zeros_like(cos, device=x.device)
        one_hot.scatter_(1, y.view(-1,1), 1.0)
        logits = (one_hot * phi + (1.0 - one_hot) * cos) * self.s
        return logits

# ── 2️⃣ Reload your classifier head to get soft-max predictions ───
ck = torch.load('/kaggle/working/effb0_arcface.pt', map_location=device)

backbone = timm.create_model('efficientnet_b0', pretrained=False,
                             num_classes=0, global_pool='avg').to(device)
proj     = nn.Linear(backbone.num_features, 512, bias=False).to(device)
arc      = ArcFace(512, NUM_CLASSES).to(device)

backbone.load_state_dict(ck['backbone'])
proj.load_state_dict(ck['proj'])
arc.load_state_dict( ck['arc'] )

backbone.eval(); proj.eval(); arc.eval()

@torch.no_grad()
def softmax_vec(x):
    feats  = proj(backbone(x))
    dummy  = torch.zeros(len(x), dtype=torch.long, device=device)
    logits = arc(feats, dummy)
    return torch.softmax(logits, dim=1)

# ── 3️⃣ Precompute soft-max on your val set ───────────────────────
pvecs = []
for imgs, _ in val_dl:
    pvecs.append( softmax_vec(imgs.to(device)).cpu() )
pvecs_val = torch.cat(pvecs)  # [N_val, NUM_CLASSES]

# ── 4️⃣ Macro-accuracy helper ───────────────────────────────────
def macro_acc(true, pred):
    classes = sorted(set(true))
    return np.mean([
        np.mean([t==p for t,p in zip(true,pred) if t==c])
        for c in classes
    ])

# (we assume the following variables are in scope from Cell 4):
#   ref_norm    # torch.Tensor [N_ref, 512]
#   ref_labels  # list[str] of length N_ref
#   best_tau    # float from Cell 4
#   val_embs    # torch.Tensor [N_val, 512]
#   val_true    # list[str] of length N_val

# ── 5️⃣ Grid-search over P_TH ∈ [0.50,0.70] and τ ±0.03 ───────────
P_grid = np.arange(0.50, 0.71, 0.02)
T_grid = np.arange(best_tau-0.03, best_tau+0.031, 0.01)

best_pair, best_ma = None, -1
print(f"Searching P×τ grid with {len(P_grid)}×{len(T_grid)} combos…")

for P_TH, τ in product(P_grid, T_grid):
    preds = []
    for prob_vec, emb in zip(pvecs_val, val_embs):
        top_p, top_i = prob_vec.max(0)
        if top_p >= P_TH:
            preds.append(employee_ids[int(top_i)])
        else:
            sims = ref_norm @ nn.functional.normalize(emb, dim=0)
            j    = int(sims.argmax())
            preds.append(ref_labels[j] if sims[j] >= τ else "unknown")
    m = macro_acc(val_true, preds)
    if m > best_ma:
        best_ma, best_pair = m, (P_TH, τ)
    print(f"P={P_TH:.2f}, τ={τ:.2f} → Val Macro-Acc {m:.4f}")

print(f"\n▶ BEST thresholds: P_TH = {best_pair[0]:.2f}, τ = {best_pair[1]:.2f}   (Val Macro-Acc = {best_ma:.4f})")


In [ ]:
# ================================================================
# Cell 5 : hybrid soft-max + gallery → submission.csv
# ================================================================
import torch
import pandas as pd
import timm
from PIL import Image

device      = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = len(employee_ids)
CKPT        = '/kaggle/working/effb0_arcface.pt'

# ── 1️⃣ Plug in your best thresholds ────────────────────────────
P_TH = best_pair[0]   # from Cell 4B
C_TH = best_pair[1]   # from Cell 4B

# ── 2️⃣ Reload backbone + ArcFace head ─────────────────────────
# (ArcFace class must already be defined in your notebook)
backbone = timm.create_model('efficientnet_b0', pretrained=False,
                             num_classes=0, global_pool='avg').to(device)
proj     = torch.nn.Linear(backbone.num_features, 512, bias=False).to(device)
arc      = ArcFace(512, NUM_CLASSES).to(device)

ck = torch.load(CKPT, map_location=device)
backbone.load_state_dict( ck['backbone'] )
proj.load_state_dict(     ck['proj']     )
arc.load_state_dict(      ck['arc']      )

backbone.eval(); proj.eval(); arc.eval()

# ── 3️⃣ Define helper functions ────────────────────────────────
@torch.no_grad()
def softmax_vec(x):
    feats  = proj(backbone(x))
    dummy  = torch.zeros(len(x), dtype=torch.long, device=device)
    logits = arc(feats, dummy)
    return torch.softmax(logits, dim=1)

@torch.no_grad()
def embed(timg):
    t = val_tfms(timg).unsqueeze(0).to(device)
    return proj(backbone(t)).squeeze().cpu()

# ── 4️⃣ Inference loop ─────────────────────────────────────────
rows = []
for img_path in sorted(TEST_IMG_DIR.glob('*.jpg')):
    pil = Image.open(img_path).convert('RGB')
    # classifier branch with softmax
    x      = val_tfms(pil).unsqueeze(0).to(device)
    pvec   = softmax_vec(x)[0].cpu()
    top_p, top_i = pvec.max(0)

    # gallery branch with cosine similarity
    emb    = embed(pil)
    emb_n  = torch.nn.functional.normalize(emb, dim=0)
    sims   = ref_norm @ emb_n         # ref_norm & ref_labels from Cell 4
    j      = int(sims.argmax())

    # hybrid decision
    if top_p >= P_TH:
        pred = employee_ids[int(top_i)]
    elif sims[j] >= C_TH:
        pred = ref_labels[j]
    else:
        pred = "unknown"

    rows.append((img_path.name, pred))

# ── 5️⃣ Write submission.csv ───────────────────────────────────
sub = pd.DataFrame(rows, columns=['image_name', 'employee_id'])
sub.to_csv('/kaggle/working/submission.csv', index=False)
print("✅ submission.csv saved:", sub.shape)
display(sub.head())


In [ ]:
# requirements.txt
# torch==2.3.0
# torchvision==0.19.0
# timm==1.0.6
# torchmetrics==1.4.0
# insightface==0.7.3
# scikit-learn==1.5.0
